In [1]:
import pandas as pd
import json
import collections
import scipy.io
import numpy as np
import mpu

In [146]:
bus = scipy.io.loadmat('MATPOWER/bus.mat')['bus']
power_demand = bus[:, 2]
df = pd.read_csv("GIS/CATS_gens.csv")
df = df[df['Pmax'] != 0.0].to_numpy()
#df = df.drop_duplicates(subset=['PlantCode','GenID']).to_numpy()

In [147]:
gen_emission = pd.read_excel("Book1.xlsx")
power_plants = pd.read_csv("Power_Plants_in_the_U.S..csv")
gen_emission = gen_emission[gen_emission['Plant state abbreviation'] == 'CA'].dropna()
gen_emission = gen_emission.rename(columns={'DOE/EIA ORIS plant or facility code': 'Plant_Code'})
gen_emission = power_plants.merge(gen_emission, on='Plant_Code', how='inner')
gen_emission = gen_emission.drop(['Plant latitude', 'Plant longitude', 'Plant state abbreviation', 'Plant name', 'State'], axis=1)
gen_emission.groupby(['PrimSource'])['Plant annual CO2 total output emission rate (lb/MWh)'].mean()

PrimSource
batteries            0.000000
biomass             76.203671
coal              1149.018000
geothermal         166.657414
hydroelectric        0.000000
natural gas       1255.016816
other              884.041200
petroleum         1808.541000
pumped storage       0.000000
solar              144.994400
Name: Plant annual CO2 total output emission rate (lb/MWh), dtype: float64

In [148]:
type_to_emission = collections.defaultdict(float)
type_to_emission['Conventional Hydroelectric'] = 0
type_to_emission['Hydroelectric Pumped Storage'] = 0
type_to_emission['Petroleum Liquids'] = 1808.5
type_to_emission['Natural Gas Internal Combustion Engine'] = 1255
type_to_emission['Natural Gas Fired Combined Cycle'] = 1255
type_to_emission['Natural Gas Steam Turbine'] = 1255
type_to_emission['Natural Gas Fired Combustion Turbine'] = 1255
type_to_emission['Nuclear'] = 0
type_to_emission['Geothermal'] = 166.7
type_to_emission['Onshore Wind Turbine'] = 0
type_to_emission['Other Waste Biomass'] = 76
type_to_emission['Wood/Wood Waste Biomass'] = 76
type_to_emission['Landfill Gas'] = 1255
type_to_emission['Solar Photovoltaic'] = 145
type_to_emission['Solar Thermal without Energy Storage'] = 145
type_to_emission['Conventional Steam Coal'] = 1149
type_to_emission['Other Gases'] = 1255
type_to_emission['Batteries'] = 0
type_to_emission['Petroleum Coke'] = 1808.5
type_to_emission['Municipal Solid Waste'] = 884
type_to_emission['Other Natural Gas'] = 1255
type_to_emission['IMPORT'] = 884
type_to_emission['Synchronous Condenser'] = 884

In [149]:
branch_ = scipy.io.loadmat('MATPOWER/branch.mat')['brach']
branch_from_bus = list(map(int, branch_[:, 0]-1))
branch_to_bus = list(map(int, branch_[:, 1]-1))

In [150]:
f = open("pf_solution.json")
sol = json.load(f)
gen = []
bus = []
branch = []
gen_cost = [0]*2149
power_generation = [0]*2149
branch_power_to = [0]*10823
branch_power_from = [0]*10823
for line, val in sol['solution']['gen'].items():
    if val['pg'] != 0.0:
        gen_cost[int(line)-1] = val['pg_cost']*100
        power_generation[int(line)-1] = val['pg']*100

for line, val in sol['solution']['branch'].items():
    branch_power_from[int(line)-1] = val['pf']*100
    branch_power_to[int(line)-1] = val['pt']*100

f.close()

In [151]:
power_generation = [0]*2149
carbon_emission = [0]*2149
gen_cost = [0]*2149
gen = [0]*2149
for i, g in enumerate(gen_):
    power_generation[g-1] = power_generation_[i]
    gen_cost[g-1] = gen_cost_[i]
    gen[g-1] = df[i][2]
    carbon_emission[g-1] = type_to_emission[df[i][3]]

In [153]:
graph = collections.defaultdict(list) # from: (to, line)
graph_reverse = collections.defaultdict(list) # to: (from, line)
branch_power_flow = collections.defaultdict(float)
for i, j in enumerate(branch):
    
    if branch_power_to[i] > 0:
        graph[branch_from_bus[i]].append((branch_to_bus[i], j))
        graph_reverse[branch_to_bus[i]].append((branch_from_bus[i], j))
    else:
        graph[branch_to_bus[i]].append((branch_from_bus[i], j))
        graph_reverse[branch_from_bus[i]].append((branch_to_bus[i], j))
    branch_power_flow[j] = abs(branch_power_to[i])

In [154]:
num_bus = len(bus)
num_branch = len(branch)
num_gen = len(gen)
print("Number of transmission lines and transformers: " + str(num_branch))
print("Number of buses: " + str(num_bus))
print("Number of generators: " + str(num_gen))

Number of transmission lines and transformers: 10823
Number of buses: 8870
Number of generators: 2149


In [155]:
line_to_gen = collections.defaultdict(set)
node_to_gen = collections.defaultdict(set)
def dfs(g, n, visited):
    if n in visited: return
    visited.add(n)
    node_to_gen[n].add(g)
    for nei, line in graph[n]:
        line_to_gen[line].add(g)
        dfs(g, nei, visited)
for i, g in enumerate(gen):
    visited = set()
    dfs(g, g, visited)

In [156]:
line_prop_mat=np.zeros((num_gen, num_branch), dtype=float)
bus_prop_mat=np.zeros((num_gen, num_bus), dtype=float)
# initially, if node_to_gen only has 1 generator, bus_prop_mat start with 1
for k, v in node_to_gen.items():
    if len(v) == 1:
        idx = list(gen).index(list(v)[0])
        bus_prop_mat[idx][int(k)] = 1.0
for k, v in line_to_gen.items():
    if len(v) == 1:
        idx = list(gen).index(list(v)[0])
        line_prop_mat[idx][int(k)] = 1.0

In [157]:
# visit nodes in topological order
# step 1: calculate bus_prop based on in flowing lines_prop
# step 2: calculate line_prop of out flowing lines
in_degree = collections.defaultdict(int)
for i, v in graph_reverse.items():
    in_degree[i] = len(v)
q = [] # list of nodes with no inflow
topo_order = []
for g in gen:
    if in_degree[g] == 0:
        q.append(g)
while q:
    cur = int(q.pop(0))
    if len(node_to_gen[cur]) > 1:
        out_total = power_demand[cur] + 0.0001
        for nei, out_line in graph[cur]:
            out_total += branch_power_flow[out_line]
        
        for g in node_to_gen[cur]:
            idx = list(gen).index(g)
            if cur == g:
                bus_prop_mat[idx][cur] = power_generation[idx]/out_total
            else:
                for nei, in_line in graph_reverse[cur]:
                    bus_prop_mat[idx][cur] += branch_power_flow[in_line]*line_prop_mat[idx][in_line]/out_total
        for g in node_to_gen[cur]:
            idx = list(gen).index(g)
            for nei, out_line in graph[cur]:
                line_prop_mat[idx][out_line] = bus_prop_mat[idx][cur]

    topo_order.append(cur)
    for nei, line in graph[cur]:
        in_degree[nei] -= 1
        if in_degree[nei] == 0:
            q.append(nei)

In [143]:
print(graph)

defaultdict(<class 'list'>, {2409: [(0, 4304), (19, 5422), (2410, 5734)], 1617: [(2410, 3935)], 1616: [(2410, 8596)], 2411: [(1616, 10467), (3954, 1126)], 2359: [(2358, 2243)], 2425: [(2030, 8169)], 5592: [(6723, 1881), (2412, 5425)], 6723: [(2425, 4209)], 8020: [(2413, 10251), (8019, 10464)], 2445: [(2414, 2923)], 2416: [(2033, 10600), (2435, 6753)], 2415: [(2435, 9571), (8020, 2631), (8020, 10121)], 2417: [(2416, 1907), (8020, 4734)], 2033: [(1866, 599), (1866, 4088), (2445, 561), (2445, 1825)], 1865: [(2439, 2491), (2454, 9842)], 2418: [(2439, 7413)], 2419: [(1865, 5944), (2439, 4793)], 2420: [(5704, 10249), (2034, 10388), (5580, 5125)], 5704: [(2418, 228), (5703, 5014)], 2421: [(2422, 3697)], 2422: [(2423, 2590)], 2424: [(2423, 8975), (2031, 5031), (2413, 2579)], 2361: [(2425, 8989)], 6725: [(2426, 5551), (2427, 3991), (6724, 5531)], 2427: [(2428, 2562)], 2428: [(2429, 3215)], 2032: [(2429, 8070)], 2430: [(2030, 6780), (2431, 4652)], 2431: [(2032, 1880)], 2432: [(1866, 9366)], 2434

In [159]:
sum(sum(bus_prop_mat))

14537725.168631395

In [113]:
avg_carbon_emission_rate_node = carbon_emission @ bus_prop_mat

In [116]:
avg_carbon_emission_rate_node[80]

1255.0